# 基于截断策略的机器阅读理解任务实现

## Step1 导入相关包

In [1]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator

## Step2 数据集加载

In [6]:
# 如果可以联网，直接使用load_dataset进行加载
#datasets = load_dataset("cmrc2018", cache_dir="data")
# 如果无法联网，则使用下面的方式加载数据集
datasets = DatasetDict.load_from_disk("mrc_data")
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [3]:
datasets["train"][0]

{'id': 'TRAIN_186_QUERY_0',
 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
 'question': '范廷颂是什么时候被任为主教的？',
 'answers': {'text': ['1963年'], 'answer_start': [30]}}

## Step3 数据预处理

In [26]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
#tokenizer

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /hfl/chinese-macbert-base/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000020928FD39D0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 7ce07b4d-8275-4f11-a59c-bb88a0b22382)')' thrown while requesting HEAD https://huggingface.co/hfl/chinese-macbert-base/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].


In [7]:
sample_dataset = datasets["train"].select(range(10))
#从训练集中选择前10个样本创建一个小的子集sample_dataset。

In [8]:
'''将问题(question)和上下文(context)进行tokenize处理
return_offsets_mapping=True:
    返回一个列表，每个元素是一个元组 (start, end)左闭右开
    star:该token在原始文本中的起始字符位置
    end:该token在原始文本中的结束字符位置

'''
tokenized_examples = tokenizer(text=sample_dataset["question"],
                        text_pair=sample_dataset["context"],
                               return_offsets_mapping=True,
                               max_length=384, truncation="only_second", padding="max_length")
tokenized_examples.keys()

'''
offset_mapping显示的是字符级别的定位信息，我们训练模型让模型得出答案是需要token级别的答案位置。

token级别  VS  字符级别
"小明在北京上班"
0 1 2 3 4 5 6 7  (字符索引)
字符级别的定位：‘北京’在(3,5)（因为左闭右开）
token级别的定位：‘北京’在3-4（对应token3代表start_pos和token4代表end_pos）
'''

'\noffset_mapping显示的是字符级别的定位信息，我们训练模型让模型得出答案是需要token级别的答案位置。\n\ntoken级别  VS  字符级别\n"小明在北京上班"\n0 1 2 3 4 5 6 7  (字符索引)\n字符级别的定位：‘北京’在(3,5)（因为左闭右开）\ntoken级别的定位：‘北京’在3-4（对应token3代表start_pos和token4代表end_pos）\n'

In [9]:
print(tokenized_examples["offset_mapping"][0], len(tokenized_examples["offset_mapping"][0]))
#前一段（0，1）到（14，15）是问题question部分

[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), (

In [10]:
offset_mapping = tokenized_examples.pop("offset_mapping")
'''
pop() 方法的作用
    获取并移除：从字典中取出指定键的值，同时从原字典中删除该键值对
    返回值：返回被移除的
执行前
tokenized_examples = {
    'input_ids': [...],
    'attention_mask': [...],
    'offset_mapping': [...]  # 执行后不再包含offset_mapping
}
'''


"\npop() 方法的作用\n    获取并移除：从字典中取出指定键的值，同时从原字典中删除该键值对\n    返回值：返回被移除的\n执行前\ntokenized_examples = {\n    'input_ids': [...],\n    'attention_mask': [...],\n    'offset_mapping': [...]  # 执行后不再包含offset_mapping\n}\n"

In [11]:
'''此处为小批量测试，最终是需要从offset_mapping里面得到答案的位置，从而转化成答案token
idx是每一个offset元组的索引，从0开始，用于给token定位，第几个索引就是token几。但是一个元组可能跨含多个数值例如hello（0，5）但是只占一个索引
'''
for idx, offset in enumerate(offset_mapping):
    answer = sample_dataset[idx]["answers"]
    '''sample_dataset是前面从预训练样本中取得小部分样本（结构从前面打印的dataset[0]可以看到），取其中的名为'answers'的部分。这里的idx代表的是对10个sample_dataset的索引（第1个sample_dataset，第二个sample_dataset...)'''
    start_char = answer["answer_start"][0]
    '''
    通常选取第一个答案作为训练目标，所以这里起始字符位置在answer_start的第一位
    '''
    end_char = start_char + len(answer["text"][0])
    # 定位答案在token中的起始位置和结束位置
    # 一种策略，我们要拿到context的起始和结束，然后从左右两侧向答案逼近

    context_start = tokenized_examples.sequence_ids(idx).index(1)
    '''context_start是tokenized_examples里面idx中
    在sequence_ids列表中查找第一个值为1的索引，这表示上下文的开始位置（第一个上下文token）
    '''
    context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1
    '''
    从context_start的index位置开始找，第一个出现None的位置，再减1，就是上下文结束context_end的位置，是token索引
    list.index(value, start_index, end_index)
    '''

    # 判断答案是否在context中——双向边界处理
    '''
    context_end是一个整数，表示当前片段中上下文部分的最后一个token在token序列中的索引。
    offset[context_end] 就是context_end这个索引对应的偏移量元组，例如(95, 100)。
    offset[context_end][1] 就是这个元组的第二个元素，即结束位置。
    '''
    if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
        start_token_pos = 0
        end_token_pos = 0
        '''
        如果answer的开始字符 比上下文最后一个token 对应的offset元组的 最后一个元素还大（说明答案在当前片段之后（被截断了））
        如果任一条件成立：答案不在当前片段，将位置设为(0,0)
        '''
    else:
        token_id = context_start
        while token_id <= context_end and offset[token_id][0] < start_char:
            token_id += 1
        start_token_pos = token_id
        '''token_id是一个可移动的索引块，从文段开始位置向后移动
        当索引值在文段内，并且其offset元组的第一个元素尚未达到answer的开始字符时。一直向后移动一位
        停止条件：找到第一个起始位置 ≥ 答案起始位置的token
        原理：这个token就是答案的起始token

        token需要+1，因为token左闭右开
        '''

        token_id = context_end
        while token_id >= context_start and offset[token_id][1] > end_char:
            token_id -=1
        end_token_pos = token_id
        
    print(answer, start_char, end_char, context_start, context_end, start_token_pos, end_token_pos)
    print("token answer decode:", tokenizer.decode(tokenized_examples["input_ids"][idx][start_token_pos: end_token_pos + 1]))

{'text': ['1963年'], 'answer_start': [30]} 30 35 17 510 47 48
token answer decode: 1963 年
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 510 53 70
token answer decode: 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理
{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 510 100 124
token answer decode: 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生
{'text': ['1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理'], 'answer_start': [548]} 548 598 17 510 0 0
token answer decode: [CLS]
{'text': ['范廷颂于2009年2月22日清晨在河内离世'], 'answer_start': [759]} 759 780 12 510 0 0
token answer decode: [CLS]
{'text': ['《全美超级模特儿新秀大赛》第十季'], 'answer_start': [26]} 26 42 21 510 47 62
token answer decode: 《 全 美 超 级 模 特 儿 新 秀 大 赛 》 第 十 季
{'text': ['有前途的新面孔'], 'answer_start': [247]} 247 254 20 510 232 238
token answer decode: 有 前 途 的 新 面 孔
{'text': ['《Jet》、《东方日报》、《Elle》等'], 'answer_start': [706]} 706 726 20 510 0 0
token answer decode: [CLS]
{'text': ['售货员'], 'answer_start': [202]}

In [12]:
def process_func(examples):
    tokenized_examples = tokenizer(text=examples["question"],
                               text_pair=examples["context"],
                               return_offsets_mapping=True,
                               max_length=384, truncation="only_second", padding="max_length")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for idx, offset in enumerate(offset_mapping):
        answer = examples["answers"][idx]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        # 定位答案在token中的起始位置和结束位置
        # 一种策略，我们要拿到context的起始和结束，然后从左右两侧向答案逼近
        context_start = tokenized_examples.sequence_ids(idx).index(1)
        context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1
        # 判断答案是否在context中
        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            start_token_pos = 0
            end_token_pos = 0
        else:
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -=1
            end_token_pos = token_id
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
    
    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

In [13]:
tokenied_datasets = datasets.map(process_func, batched=True, remove_columns=datasets["train"].column_names)
#移除原始数据的所有列，因为预处理后的数据已经有了新的结构
tokenied_datasets

Map:   0%|          | 0/10142 [00:00<?, ? examples/s]

Map:   0%|          | 0/3219 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1002
    })
})

## Step4 加载模型

In [14]:
model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step5 配置TrainingArguments

In [15]:
args = TrainingArguments(
    output_dir="models_for_qa",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    num_train_epochs=3
)

## Step6 配置Trainer

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenied_datasets["train"],
    eval_dataset=tokenied_datasets["validation"],
    data_collator=DefaultDataCollator()
)

## Step7 模型训练

In [17]:
trainer.train()

D:\Anaconda\envs\llm-project\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss
1,1.386500,1.063317
2,0.965000,1.062937
3,0.639300,1.178824


TrainOutput(global_step=951, training_loss=1.1234789342158225, metrics={'train_runtime': 7169.2571, 'train_samples_per_second': 4.244, 'train_steps_per_second': 0.133, 'total_flos': 5962661340337152.0, 'train_loss': 1.1234789342158225, 'epoch': 3.0})

## Step8 模型预测

In [18]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

In [25]:
pipe(question="小陈在哪里上学？", context="小陈上学的地方在龙岩学院。")

{'score': 0.654445230960846, 'start': 8, 'end': 12, 'answer': '龙岩学院'}